In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import warnings
import os
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
# scanpy 설정
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [3]:
os.chdir('C:/Users/laugh/Desktop/250515/liver_new_target_analysis')
project_path = os.getcwd()
data_path = os.path.join(project_path, "data")
os.chdir(data_path)
file_path = os.path.join(data_path, "processed.h5ad")
print(file_path)

C:\Users\laugh\Desktop\250515\liver_new_target_analysis\data\processed.h5ad


In [4]:
# 데이터 로드
adata = sc.read_h5ad('processed.h5ad')

print(f"Total cells: {adata.n_obs}")
print(f"Total genes: {adata.n_vars}")

Total cells: 71914
Total genes: 25479


In [5]:
# 1. T/NK 세포만 추출
t_cells = adata[adata.obs['celltype'] == 'T/NK'].copy()
print(f"\nT/NK cells: {t_cells.n_obs}")
print(f"T/NK cell distribution by site:")
print(t_cells.obs['site'].value_counts())


T/NK cells: 25590
T/NK cell distribution by site:
site
Normal    17691
Tumor      6830
PVTT        584
Lymph       485
Name: count, dtype: int64


In [6]:
# 2. 알려진 T cell marker 유전자 정의 (제외할 유전자들)
known_markers = {
    'CD3D', 'CD3E', 'CD3G', 'CD2', 'CD5', 'CD7',  # Pan-T
    'CD4', 'IL7R', 'CCR7', 'SELL', 'TCF7', 'LEF1',  # CD4+
    'CD8A', 'CD8B', 'CCL5', 'NKG7', 'GZMA', 'GZMB', 'PRF1',  # CD8+
    'FOXP3', 'IL2RA', 'CTLA4', 'IKZF2', 'TNFRSF4', 'TNFRSF18',  # Treg
    'TBX21', 'IFNG', 'TNF', 'IL2', 'CXCR3', 'CCR5',  # Th1
    'GATA3', 'IL4', 'IL5', 'IL13', 'CCR4',  # Th2
    'RORC', 'IL17A', 'IL17F', 'IL21', 'IL22', 'CCR6',  # Th17
    'PDCD1', 'TIGIT', 'LAG3', 'HAVCR2', 'TOX', 'ENTPD1',  # Exhausted
    'KLRB1', 'KLRD1', 'KLRF1', 'NCR1', 'NCR2', 'NCR3', 'NCAM1'  # NK
}

# 실제 데이터에 존재하는 알려진 마커들
known_markers_in_data = [gene for gene in known_markers if gene in t_cells.var_names]
print(f"\nKnown markers in data: {len(known_markers_in_data)}")


Known markers in data: 55


In [7]:
# 3. 알려진 마커를 제외한 유전자들로 novel marker 탐색
novel_candidates = [gene for gene in t_cells.var_names if gene not in known_markers]
print(f"Novel candidate genes: {len(novel_candidates)}")


Novel candidate genes: 25424


In [8]:
# 4. 종양 vs 정상 조직에서 차등 발현 유전자 분석
def find_novel_markers_de_analysis(adata, condition_col='site', 
                                  condition1='Tumor', condition2='Normal',
                                  min_fold_change=1.5, max_pval=0.01):
    """차등 발현 분석을 통한 novel marker 발굴"""
    
    print(f"\n=== Differential Expression Analysis: {condition1} vs {condition2} ===")
    
    # 조건별 세포 수 확인
    tumor_cells = adata[adata.obs[condition_col] == condition1]
    normal_cells = adata[adata.obs[condition_col] == condition2]
    
    print(f"{condition1} cells: {tumor_cells.n_obs}")
    print(f"{condition2} cells: {normal_cells.n_obs}")
    
    if tumor_cells.n_obs == 0 or normal_cells.n_obs == 0:
        print("Insufficient cells for comparison")
        return pd.DataFrame()
    
    results = []
    
    print(f"Processing {len(novel_candidates)} genes...")
    for gene in tqdm(novel_candidates, desc="DE Analysis"):
        # 발현 데이터 추출
        tumor_expr = tumor_cells[:, gene].X.toarray().flatten()
        normal_expr = normal_cells[:, gene].X.toarray().flatten()
        
        # 발현 세포 비율 계산
        tumor_pct = np.mean(tumor_expr > 0) * 100
        normal_pct = np.mean(normal_expr > 0) * 100
        
        # 평균 발현량 계산
        tumor_mean = np.mean(tumor_expr)
        normal_mean = np.mean(normal_expr)
        
        # 통계 검정
        if tumor_mean > 0 or normal_mean > 0:
            try:
                stat, pval = stats.mannwhitneyu(tumor_expr, normal_expr, alternative='two-sided')
                fold_change = (tumor_mean + 1e-10) / (normal_mean + 1e-10)
                log2_fc = np.log2(fold_change)
            except:
                pval = 1.0
                fold_change = 1.0
                log2_fc = 0.0
            
            results.append({
                'gene': gene,
                'tumor_mean': tumor_mean,
                'normal_mean': normal_mean,
                'tumor_pct': tumor_pct,
                'normal_pct': normal_pct,
                'fold_change': fold_change,
                'log2_fold_change': log2_fc,
                'pvalue': pval,
                'significant': pval < max_pval and abs(log2_fc) > np.log2(min_fold_change)
            })
    
    df = pd.DataFrame(results)
    return df.sort_values('pvalue')

In [ ]:
# 5. 종양 vs 정상 조직 비교
de_results = find_novel_markers_de_analysis(t_cells)
de_results.to_csv('novel_markers_de_results.csv', index=False) # 내가 추가함

# 유의한 novel marker 출력
significant_novel = de_results[de_results['significant'] == True]
print(f"\nSignificant novel markers found: {len(significant_novel)}")
significant_novel.to_csv('significant_novel_markers_de_results.csv', index=False) # 내가 추가함

if len(significant_novel) > 0:
    print("\nTop 20 upregulated novel markers in tumor:")
    upregulated = significant_novel[significant_novel['log2_fold_change'] > 0].head(20)
    for _, row in upregulated.iterrows():
        print(f"  {row['gene']}: FC={row['fold_change']:.2f}, p={row['pvalue']:.2e}")
    
    print("\nTop 20 downregulated novel markers in tumor:")
    downregulated = significant_novel[significant_novel['log2_fold_change'] < 0].head(20)
    for _, row in downregulated.iterrows():
        print(f"  {row['gene']}: FC={row['fold_change']:.2f}, p={row['pvalue']:.2e}")



=== Differential Expression Analysis: Tumor vs Normal ===
Tumor cells: 6830
Normal cells: 17691
Processing 25424 genes...


DE Analysis: 100%|██████████| 25424/25424 [2:17:10<00:00,  3.09it/s]  


AttributeError: module 'pandas' has no attribute 'to_csv'

In [ ]:
# 6. 부위별 특이적 마커 분석
def find_site_specific_markers(adata, min_fold_change=2.0, max_pval=0.01):
    """각 부위별 특이적 마커 발굴"""
    
    print(f"\n=== Site-Specific Marker Analysis ===")
    
    sites = adata.obs['site'].unique()
    site_specific_markers = {}
    
    for site in tqdm(sites, desc="Site-specific analysis"):
        print(f"\nAnalyzing {site}-specific markers...")
        
        site_cells = adata[adata.obs['site'] == site]
        other_cells = adata[adata.obs['site'] != site]
        
        if site_cells.n_obs < 50:  # 최소 세포 수 확인
            print(f"Skipping {site}: insufficient cells ({site_cells.n_obs})")
            continue
        
        results = []
        
        for gene in tqdm(novel_candidates, desc=f"{site} markers", leave=False):
            site_expr = site_cells[:, gene].X.toarray().flatten()
            other_expr = other_cells[:, gene].X.toarray().flatten()
            
            site_mean = np.mean(site_expr)
            other_mean = np.mean(other_expr)
            
            if site_mean > 0 or other_mean > 0:
                try:
                    stat, pval = stats.mannwhitneyu(site_expr, other_expr, alternative='two-sided')
                    fold_change = (site_mean + 1e-10) / (other_mean + 1e-10)
                    log2_fc = np.log2(fold_change)
                except:
                    pval = 1.0
                    fold_change = 1.0
                    log2_fc = 0.0
                
                if pval < max_pval and fold_change > min_fold_change:
                    results.append({
                        'gene': gene,
                        'site_mean': site_mean,
                        'other_mean': other_mean,
                        'fold_change': fold_change,
                        'log2_fold_change': log2_fc,
                        'pvalue': pval
                    })
        
        if len(results) > 0:
            site_df = pd.DataFrame(results).sort_values('pvalue')
            site_specific_markers[site] = site_df
            
            print(f"Top 10 {site}-specific markers:")
            for _, row in site_df.head(10).iterrows():
                print(f"  {row['gene']}: FC={row['fold_change']:.2f}, p={row['pvalue']:.2e}")
    
    return site_specific_markers

# 부위별 특이적 마커 분석
site_markers = find_site_specific_markers(t_cells)

# 부위별 마커 저장
for site, df in site_markers.items():
    filename = f"site_specific_markers/markers_{site}.csv"
    df.to_csv(filename, index=False)
    print(f"✅ {site} 마커 저장 완료: {filename}")


=== Site-Specific Marker Analysis ===


Site-specific analysis:   0%|          | 0/4 [00:00<?, ?it/s]


Analyzing Tumor-specific markers...


Site-specific analysis:  25%|██▌       | 1/4 [2:16:22<6:49:08, 8182.81s/it]

Top 10 Tumor-specific markers:
  GALE: FC=3.98, p=0.00e+00
  PPP1R16A: FC=5.36, p=0.00e+00
  PLS3: FC=4.55, p=0.00e+00
  PGRMC1: FC=2.63, p=0.00e+00
  HCFC1R1: FC=2.70, p=0.00e+00
  FNBP1L: FC=4.60, p=0.00e+00
  PAGE5: FC=4.52, p=0.00e+00
  COL4A2: FC=5.31, p=0.00e+00
  EPHX1: FC=4.98, p=0.00e+00
  GDF15: FC=4.72, p=0.00e+00

Analyzing Normal-specific markers...


...

Top 10 Normal-specific markers:
  CD160: FC=15.05, p=0.00e+00
  DUSP2: FC=2.33, p=0.00e+00
  IGHA1: FC=4.23, p=0.00e+00
  GPR65: FC=3.47, p=0.00e+00
  ISG20: FC=2.16, p=0.00e+00
  PTPRC: FC=2.16, p=0.00e+00
  CORO1A: FC=2.05, p=0.00e+00
  AKNA: FC=4.67, p=0.00e+00
  GZMM: FC=5.49, p=0.00e+00
  SYTL3: FC=4.68, p=0.00e+00

Analyzing PVTT-specific markers...


Site-specific analysis:  75%|███████▌  | 3/4 [6:09:21<1:56:58, 7018.27s/it]

Top 10 PVTT-specific markers:
  PODXL2: FC=20.44, p=0.00e+00
  MISP: FC=13.94, p=0.00e+00
  HMGA2: FC=29.95, p=0.00e+00
  VGF: FC=30.54, p=0.00e+00
  C9orf116: FC=14.78, p=0.00e+00
  PGAM2: FC=19.72, p=0.00e+00
  TGFBR3L: FC=11.91, p=0.00e+00
  MAP7D2: FC=40.89, p=0.00e+00
  SALL4: FC=33.65, p=0.00e+00
  SPP1: FC=4.06, p=0.00e+00

Analyzing Lymph-specific markers...


Site-specific analysis: 100%|██████████| 4/4 [7:36:47<00:00, 6851.86s/it]  

Top 10 Lymph-specific markers:
  SLC6A15: FC=168.99, p=0.00e+00
  C19orf33: FC=14.10, p=0.00e+00
  KLK1: FC=42.65, p=0.00e+00
  SPINK1: FC=11.01, p=0.00e+00
  ADGRV1: FC=27.85, p=0.00e+00
  ELFN1-AS1: FC=37.04, p=0.00e+00
  REG1A: FC=44.56, p=0.00e+00
  C5orf17: FC=69.84, p=0.00e+00
  AGR2: FC=26.69, p=0.00e+00
  AGR3: FC=34.69, p=0.00e+00


In [15]:
# 7. 기계학습을 통한 특징 선택
def ml_feature_selection(adata, target_col='site', n_features=50):
    """기계학습을 통한 중요한 특징 선택"""
    
    print(f"\n=== Machine Learning Feature Selection ===")
    
    # 데이터 준비
    X = adata[:, novel_candidates].X.toarray()
    y = adata.obs[target_col].values
    
    # 특징 선택 - F-score 기반
    print("Computing F-scores for feature selection...")
    selector = SelectKBest(f_classif, k=min(n_features, len(novel_candidates)))
    print(f"Selected k (number of features): {selector.k}")
    X_selected = selector.fit_transform(X, y)
    
    # 선택된 특징들
    selected_features = np.array(novel_candidates)[selector.get_support()]
    feature_scores = selector.scores_[selector.get_support()]
    
    # Random Forest로 특징 중요도 계산
    print("Training Random Forest for feature importance...")
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_selected, y)
    
    # 결과 정리
    feature_importance = pd.DataFrame({
        'gene': selected_features,
        'f_score': feature_scores,
        'rf_importance': rf.feature_importances_
    }).sort_values('rf_importance', ascending=False)
    
    print(f"Top 20 features by Random Forest importance:")
    for _, row in feature_importance.head(20).iterrows():
        print(f"  {row['gene']}: RF={row['rf_importance']:.4f}, F-score={row['f_score']:.2f}")
    
    return feature_importance

# 기계학습 특징 선택
ml_features = ml_feature_selection(t_cells)
ml_features.to_csv('ml_selected_features.csv', index=False) # 내가 추가함


=== Machine Learning Feature Selection ===
Computing F-scores for feature selection...
Selected k (number of features): 50
Training Random Forest for feature importance...
Top 20 features by Random Forest importance:
  APOA2: RF=0.1076, F-score=3466.71
  SPP1: RF=0.0766, F-score=2269.68
  APOE: RF=0.0565, F-score=2920.53
  AMBP: RF=0.0497, F-score=2633.28
  LGALS1: RF=0.0492, F-score=2035.43
  RPL17: RF=0.0475, F-score=1806.17
  SERPINA1: RF=0.0416, F-score=3449.28
  AGT: RF=0.0358, F-score=2452.61
  RARRES2: RF=0.0317, F-score=2227.49
  B2M: RF=0.0314, F-score=2680.05
  VTN: RF=0.0305, F-score=2328.62
  SRGN: RF=0.0298, F-score=2709.70
  HSPB1: RF=0.0292, F-score=2460.56
  APOC1: RF=0.0268, F-score=2609.10
  HLA-B: RF=0.0235, F-score=1767.05
  RPS29: RF=0.0233, F-score=1835.03
  PRAP1: RF=0.0214, F-score=1716.78
  CXCR4: RF=0.0211, F-score=2974.95
  MGST1: RF=0.0197, F-score=2086.28
  SPINK1: RF=0.0182, F-score=2401.59


In [ ]:
# # 8. 환자별 예후와 연관된 마커 분석
# def find_prognostic_markers(adata, stage_col='stage', virus_col='virus'):
#     """환자 예후와 연관된 마커 발굴"""
    
#     print(f"\n=== Prognostic Marker Analysis ===")
    
#     prognostic_results = []
    
#     # 병기별 분석
#     if stage_col in adata.obs.columns:
#         stages = adata.obs[stage_col].unique()
#         print(f"Available stages: {stages}")
        
#         print("Analyzing stage progression markers...")
#         #for gene in tqdm(novel_candidates[:1000], desc="Stage analysis"): # 계산 시간 고려하여 상위 1000개 유전자만
#         for gene in tqdm(novel_candidates, desc="Stage analysis"):
#             stage_expr = []
#             for stage in stages:
#                 stage_cells = adata[adata.obs[stage_col] == stage]
#                 if stage_cells.n_obs > 0:
#                     expr = np.mean(stage_cells[:, gene].X.toarray())
#                     stage_expr.append(expr)
            
#             if len(stage_expr) > 1:
#                 # 병기와 발현의 상관관계
#                 try:
#                     corr, pval = stats.spearmanr(range(len(stages)), stage_expr)
#                     if abs(corr) > 0.3 and pval < 0.05:
#                         prognostic_results.append({
#                             'gene': gene,
#                             'correlation': corr,
#                             'pvalue': pval,
#                             'type': 'stage_progression'
#                         })
#                 except:
#                     pass
    
#     # 바이러스 감염 여부별 분석
#     if virus_col in adata.obs.columns:
#         virus_types = adata.obs[virus_col].unique()
#         print(f"Available virus types: {virus_types}")
        
#         print("Analyzing virus-associated markers...")
#         for gene in tqdm(novel_candidates[:1000], desc="Virus analysis"):
#             virus_results = []
#             for virus in virus_types:
#                 virus_cells = adata[adata.obs[virus_col] == virus]
#                 if virus_cells.n_obs > 10:
#                     expr = np.mean(virus_cells[:, gene].X.toarray())
#                     virus_results.append((virus, expr))
            
#             if len(virus_results) > 1:
#                 # 바이러스 타입별 발현 차이
#                 exprs = [x[1] for x in virus_results]
#                 if max(exprs) / (min(exprs) + 1e-10) > 2:  # 2배 이상 차이
#                     prognostic_results.append({
#                         'gene': gene,
#                         'max_expr': max(exprs),
#                         'min_expr': min(exprs),
#                         'fold_change': max(exprs) / (min(exprs) + 1e-10),
#                         'type': 'virus_associated'
#                     })
    
#     return prognostic_results

# # 예후 마커 분석
# prognostic_markers = find_prognostic_markers(t_cells)

# if len(prognostic_markers) > 0:
#     print(f"\nPrognostic markers found: {len(prognostic_markers)}")
#     prog_df = pd.DataFrame(prognostic_markers)
#     prog_df.to_csv('prognostic_markers.csv', index=False) # 내가 추가함
    
#     # 병기 진행 관련 마커
#     stage_markers = prog_df[prog_df['type'] == 'stage_progression']
#     if len(stage_markers) > 0:
#         print("\nStage progression markers:")
#         for _, row in stage_markers.head(10).iterrows():
#             print(f"  {row['gene']}: corr={row['correlation']:.3f}, p={row['pvalue']:.2e}")
    
#     # 바이러스 관련 마커
#     virus_markers = prog_df[prog_df['type'] == 'virus_associated']
#     if len(virus_markers) > 0:
#         print("\nVirus-associated markers:")
#         for _, row in virus_markers.head(10).iterrows():
#             print(f"  {row['gene']}: FC={row['fold_change']:.2f}")


In [ ]:
def find_prognostic_markers(adata, stage_col='stage', virus_col='virus'):
    """환자 예후와 연관된 마커 발굴"""
    
    print(f"\n=== Prognostic Marker Analysis ===")
    
    prognostic_results = []
    
    # 병기별 분석
    if stage_col in adata.obs.columns:
        # 병기 순서 정의 (I -> II -> IIIA -> IIIB -> IV)
        stage_order = ['I', 'II', 'IIIA', 'IIIB', 'IV']
        available_stages = adata.obs[stage_col].unique()
        
        # 실제 데이터에 있는 병기만 필터링하고 순서대로 정렬
        ordered_stages = [stage for stage in stage_order if stage in available_stages]
        
        print(f"Available stages: {available_stages}")
        print(f"Ordered stages for analysis: {ordered_stages}")
        
        if len(ordered_stages) > 1:  # 최소 2개 이상의 병기가 있어야 분석 가능
            print("Analyzing stage progression markers...")
            
            for gene in tqdm(novel_candidates, desc="Stage analysis"):
                stage_expr = []
                stage_numeric = []
                
                for i, stage in enumerate(ordered_stages):
                    stage_cells = adata[adata.obs[stage_col] == stage]
                    if stage_cells.n_obs > 0:
                        expr = np.mean(stage_cells[:, gene].X.toarray())
                        stage_expr.append(expr)
                        stage_numeric.append(i)  # 병기 순서를 숫자로 변환 (0, 1, 2, ...)
                
                if len(stage_expr) > 1:
                    # 병기 순서와 발현의 상관관계
                    try:
                        corr, pval = stats.spearmanr(stage_numeric, stage_expr)
                        if abs(corr) > 0.3 and pval < 0.05:
                            # 추가 정보: 각 병기별 평균 발현값
                            stage_expressions = {}
                            for stage, expr in zip(ordered_stages, stage_expr):
                                stage_expressions[stage] = expr
                            
                            prognostic_results.append({
                                'gene': gene,
                                'correlation': corr,
                                'pvalue': pval,
                                'type': 'stage_progression',
                                'stage_expressions': stage_expressions,
                                'direction': 'increasing' if corr > 0 else 'decreasing'
                            })
                    except:
                        pass
    
    # 바이러스 감염 여부별 분석
    if virus_col in adata.obs.columns:
        virus_types = adata.obs[virus_col].dropna().unique()  # NaN 제거
        print(f"Available virus types: {virus_types}")
        
        if len(virus_types) > 1:
            print("Analyzing virus-associated markers...")
            for gene in tqdm(novel_candidates[:1000], desc="Virus analysis"):
                virus_results = []
                for virus in virus_types:
                    virus_cells = adata[adata.obs[virus_col] == virus]
                    if virus_cells.n_obs > 10:
                        expr = np.mean(virus_cells[:, gene].X.toarray())
                        virus_results.append((virus, expr))
                
                if len(virus_results) > 1:
                    # 바이러스 타입별 발현 차이
                    exprs = [x[1] for x in virus_results]
                    if max(exprs) / (min(exprs) + 1e-10) > 2:  # 2배 이상 차이
                        virus_expr_dict = {virus: expr for virus, expr in virus_results}
                        prognostic_results.append({
                            'gene': gene,
                            'max_expr': max(exprs),
                            'min_expr': min(exprs),
                            'fold_change': max(exprs) / (min(exprs) + 1e-10),
                            'type': 'virus_associated',
                            'virus_expressions': virus_expr_dict
                        })
    
    return prognostic_results

# 예후 마커 분석
prognostic_markers = find_prognostic_markers(t_cells)

if len(prognostic_markers) > 0:
    print(f"\nPrognostic markers found: {len(prognostic_markers)}")
    prog_df = pd.DataFrame(prognostic_markers)
    
    # CSV 저장을 위해 딕셔너리 컬럼들을 개별 컬럼으로 분리
    prog_df_export = prog_df.copy()
    
    # stage_expressions 딕셔너리를 개별 컬럼으로 분리
    stage_markers = prog_df[prog_df['type'] == 'stage_progression']
    if len(stage_markers) > 0:
        stage_order = ['I', 'II', 'IIIA', 'IIIB', 'IV']
        for stage in stage_order:
            prog_df_export[f'stage_{stage}_expr'] = prog_df_export['stage_expressions'].apply(
                lambda x: x.get(stage, np.nan) if isinstance(x, dict) else np.nan
            )
    
    # virus_expressions 딕셔너리를 개별 컬럼으로 분리
    virus_markers = prog_df[prog_df['type'] == 'virus_associated']
    if len(virus_markers) > 0:
        virus_types = ['HBV', 'HCV']  # 실제 바이러스 타입에 맞게 조정
        for virus in virus_types:
            prog_df_export[f'virus_{virus}_expr'] = prog_df_export['virus_expressions'].apply(
                lambda x: x.get(virus, np.nan) if isinstance(x, dict) else np.nan
            )
    
    # 딕셔너리 컬럼들 제거
    columns_to_drop = ['stage_expressions', 'virus_expressions']
    prog_df_export = prog_df_export.drop(columns=[col for col in columns_to_drop if col in prog_df_export.columns])
    
    # CSV 파일 저장
    prog_df_export.to_csv('prognostic_markers.csv', index=False)
    print(f"\nPrognostic markers saved to 'prognostic_markers.csv'")
    print(f"CSV file contains {len(prog_df_export)} markers with detailed expression values.")
    
    # 병기 진행 관련 마커
    stage_markers = prog_df[prog_df['type'] == 'stage_progression']
    if len(stage_markers) > 0:
        print(f"\nStage progression markers found: {len(stage_markers)}")
        
        # 병기 증가에 따라 발현이 증가하는 마커들
        increasing_markers = stage_markers[stage_markers['direction'] == 'increasing']
        if len(increasing_markers) > 0:
            print(f"\nMarkers increasing with stage progression ({len(increasing_markers)}):")
            for _, row in increasing_markers.head(10).iterrows():
                print(f"  {row['gene']}: corr={row['correlation']:.3f}, p={row['pvalue']:.2e}")
        
        # 병기 증가에 따라 발현이 감소하는 마커들
        decreasing_markers = stage_markers[stage_markers['direction'] == 'decreasing']
        if len(decreasing_markers) > 0:
            print(f"\nMarkers decreasing with stage progression ({len(decreasing_markers)}):")
            for _, row in decreasing_markers.head(10).iterrows():
                print(f"  {row['gene']}: corr={row['correlation']:.3f}, p={row['pvalue']:.2e}")
    
    # 바이러스 관련 마커
    virus_markers = prog_df[prog_df['type'] == 'virus_associated']
    if len(virus_markers) > 0:
        print(f"\nVirus-associated markers found: {len(virus_markers)}")
        for _, row in virus_markers.head(10).iterrows():
            print(f"  {row['gene']}: FC={row['fold_change']:.2f}")

else:
    print("No prognostic markers found with current criteria.")

# 상세 분석을 위한 시각화 함수
def plot_stage_progression_markers(adata, markers_df, top_n=5):
    """병기 진행과 관련된 마커들의 발현 패턴 시각화"""
    
    stage_markers = markers_df[markers_df['type'] == 'stage_progression']
    if len(stage_markers) == 0:
        print("No stage progression markers to plot.")
        return
    
    # 상위 마커들 선택
    top_markers = stage_markers.nlargest(top_n, 'correlation')
    
    stage_order = ['I', 'II', 'IIIA', 'IIIB', 'IV']
    available_stages = [stage for stage in stage_order if stage in adata.obs['stage'].unique()]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, (_, marker) in enumerate(top_markers.iterrows()):
        if i >= 6:  # 최대 6개만 표시
            break
            
        gene = marker['gene']
        stage_expr = []
        
        for stage in available_stages:
            stage_cells = adata[adata.obs['stage'] == stage]
            if stage_cells.n_obs > 0:
                expr = np.mean(stage_cells[:, gene].X.toarray())
                stage_expr.append(expr)
        
        axes[i].plot(range(len(available_stages)), stage_expr, 'o-', linewidth=2, markersize=8)
        axes[i].set_title(f'{gene}\n(corr={marker["correlation"]:.3f}, p={marker["pvalue"]:.2e})')
        axes[i].set_xlabel('Stage')
        axes[i].set_ylabel('Expression')
        axes[i].set_xticks(range(len(available_stages)))
        axes[i].set_xticklabels(available_stages)
        axes[i].grid(True, alpha=0.3)
    
    # 빈 subplot 숨기기
    for i in range(len(top_markers), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.savefig('stage_progression_markers.png', dpi=300, bbox_inches='tight')
    plt.show()

# 시각화 실행
if len(prognostic_markers) > 0:
    plot_stage_progression_markers(t_cells, prog_df)


=== Prognostic Marker Analysis ===
Available stages: ['IIIA', 'IV', 'IIIB', 'II', 'I']
Categories (5, object): ['I', 'II', 'IIIA', 'IIIB', 'IV']
Ordered stages for analysis: ['I', 'II', 'IIIA', 'IIIB', 'IV']
Analyzing stage progression markers...


Stage analysis:   0%|          | 2/25424 [00:01<6:47:58,  1.04it/s]

In [ ]:
prognostic_markers.to_csv('prognostic_markers.csv', index=False)  # 내가 추가함


Available stages: ['I', 'IIIA', 'IV', 'IIIB', 'II']
Categories (5, object): ['I', 'II', 'IIIA', 'IIIB', 'IV']
                         sample  res.3    site patient stage virus  \
Cell                                                                 
HCC03T_ACTATCTTCGGCGCAT  HCC03T     15   Tumor   HCC03     I   HCV   
HCC06T_TTCTACAAGCTTCGCG  HCC06T     35   Tumor   HCC06  IIIA   HBV   
HCC10N_AAGACCTGTAAGAGAG  HCC10N     13  Normal   HCC10    IV   HBV   
HCC10L_ATCCGAACATCCGCGA  HCC10L      3   Lymph   HCC10    IV   HBV   
HCC05N_TTGCCGTAGTGGTAGC  HCC05N      2  Normal   HCC05  IIIA   NaN   
...                         ...    ...     ...     ...   ...   ...   
HCC04T_ACTGTCCTCCGTTGTC  HCC04T     24   Tumor   HCC04    II   HCV   
HCC06T_CCTTCCCGTGACCAAG  HCC06T     14   Tumor   HCC06  IIIA   HBV   
HCC06N_GGACAAGTCATCGGAT  HCC06N      5  Normal   HCC06  IIIA   HBV   
HCC01T_ATCACGACATCTGGTA  HCC01T      2   Tumor   HCC01     I   HBV   
HCC03T_GCGCGATTCACTCTTA  HCC03T      3   Tumor   

In [28]:
# 9. 결과 통합 및 우선순위 설정
def integrate_results(de_results, site_markers, ml_features, prognostic_markers):
    """모든 분석 결과를 통합하여 최종 novel marker 우선순위 설정"""
    
    print(f"\n=== Novel Marker Prioritization ===")
    
    # 각 유전자별 점수 계산
    gene_scores = {}
    
    # DE 분석 점수
    print("Integrating DE analysis scores...")
    for _, row in tqdm(de_results.iterrows(), desc="DE scores", total=len(de_results)):
        if row['significant']:
            gene = row['gene']
            score = abs(row['log2_fold_change']) * (-np.log10(row['pvalue']))
            gene_scores[gene] = gene_scores.get(gene, 0) + score
    
    # 부위별 특이성 점수
    print("Integrating site-specific scores...")
    for site, markers in tqdm(site_markers.items(), desc="Site scores"):
        for _, row in markers.iterrows():
            gene = row['gene']
            score = abs(row['log2_fold_change']) * (-np.log10(row['pvalue']))
            gene_scores[gene] = gene_scores.get(gene, 0) + score * 0.5
    
    # ML 중요도 점수
    print("Integrating ML importance scores...")
    for _, row in tqdm(ml_features.iterrows(), desc="ML scores", total=len(ml_features)):
        gene = row['gene']
        score = row['rf_importance'] * 100
        gene_scores[gene] = gene_scores.get(gene, 0) + score
    
    # 예후 관련 점수
    print("Integrating prognostic scores...")
    for marker in tqdm(prognostic_markers, desc="Prognostic scores"):
        gene = marker['gene']
        if marker['type'] == 'stage_progression':
            score = abs(marker['correlation']) * (-np.log10(marker['pvalue']))
        else:
            score = np.log2(marker['fold_change'])
        gene_scores[gene] = gene_scores.get(gene, 0) + score
    
    # 최종 우선순위
    print("Computing final ranking...")
    final_ranking = sorted(gene_scores.items(), key=lambda x: x[1], reverse=True)
    
    print("Top 30 Novel T Cell Biomarker Candidates:")
    for i, (gene, score) in enumerate(final_ranking[:30]):
        print(f"{i+1:2d}. {gene}: score={score:.3f}")
    
    return final_ranking

# 최종 결과 통합
final_candidates = integrate_results(de_results, site_markers, ml_features, prognostic_markers)



=== Novel Marker Prioritization ===
Integrating DE analysis scores...


DE scores: 100%|██████████| 24108/24108 [00:00<00:00, 46402.43it/s]


Integrating site-specific scores...


Site scores: 100%|██████████| 4/4 [00:00<00:00,  9.29it/s]


Integrating ML importance scores...


ML scores: 100%|██████████| 50/50 [00:00<00:00, 49884.68it/s]


Integrating prognostic scores...


Prognostic scores: 100%|██████████| 329/329 [00:00<?, ?it/s]

Computing final ranking...
Top 30 Novel T Cell Biomarker Candidates:
 1. CYP3A5: score=inf
 2. ARPC1A: score=inf
 3. DDAH2: score=inf
 4. UQCRC1: score=inf
 5. KNOP1: score=inf
 6. HES4: score=inf
 7. RGS1: score=inf
 8. POP7: score=inf
 9. TRIP6: score=inf
10. TM4SF5: score=inf
11. HSPA1B: score=inf
12. C2: score=inf
13. APOA1BP: score=inf
14. MRPL24: score=inf
15. PDCD5: score=inf
16. FDPS: score=inf
17. LAMTOR2: score=inf
18. LMNA: score=inf
19. GLMP: score=inf
20. HPN: score=inf
21. FXYD1: score=inf
22. CFB: score=inf
23. TFR2: score=inf
24. SORBS2: score=inf
25. GZMK: score=inf
26. NDUFC1: score=inf
27. MGST2: score=inf
28. HACD3: score=inf
29. CYP17A1: score=inf
30. GSTM3: score=inf


In [29]:
# 10. 결과 저장
print(f"\n=== Saving Results ===")
print("""
# 결과 저장
de_results.to_csv('novel_markers_DE_analysis.csv', index=False)

# 부위별 마커 저장
for site, markers in site_markers.items():
    markers.to_csv(f'novel_markers_{site}_specific.csv', index=False)

# ML 특징 저장
ml_features.to_csv('novel_markers_ML_features.csv', index=False)

# 최종 후보 저장
final_df = pd.DataFrame(final_candidates, columns=['gene', 'integrated_score'])
final_df.to_csv('novel_tcell_biomarkers_final_ranking.csv', index=False)

print("Novel biomarker discovery analysis completed!")
""")

print(f"\n=== Summary ===")
print("Novel T cell biomarker discovery analysis completed.")
print("Key analyses performed:")
print("1. Differential expression analysis (Tumor vs Normal)")
print("2. Site-specific marker identification")
print("3. Machine learning-based feature selection")
print("4. Prognostic marker analysis")
print("5. Integrated scoring and prioritization")
print(f"6. Final ranking of {len(final_candidates)} potential novel biomarkers")


=== Saving Results ===

# 결과 저장
de_results.to_csv('novel_markers_DE_analysis.csv', index=False)

# 부위별 마커 저장
for site, markers in site_markers.items():
    markers.to_csv(f'novel_markers_{site}_specific.csv', index=False)

# ML 특징 저장
ml_features.to_csv('novel_markers_ML_features.csv', index=False)

# 최종 후보 저장
final_df = pd.DataFrame(final_candidates, columns=['gene', 'integrated_score'])
final_df.to_csv('novel_tcell_biomarkers_final_ranking.csv', index=False)

print("Novel biomarker discovery analysis completed!")


=== Summary ===
Novel T cell biomarker discovery analysis completed.
Key analyses performed:
1. Differential expression analysis (Tumor vs Normal)
2. Site-specific marker identification
3. Machine learning-based feature selection
4. Prognostic marker analysis
5. Integrated scoring and prioritization
6. Final ranking of 15114 potential novel biomarkers
